In [ ]:
import scipy.io as sio
import numpy as np
from matplotlib.colors import ListedColormap
from scipy.stats import pearsonr
import seaborn as sn
import mne
import matplotlib.pyplot as plt
import pandas as pd
import os
#!pip install EDFlib-Python

%matplotlib qt
%gui qt

#str_DataPath = 'C:/Users/Sebastian/Documents/NeuroZenProcessing/Preprocessed/' 
str_DataPath = "D:/Preprocessed/" 
str_FileName_MF = str_DataPath+'Sujeto35_MF_processed.fif'  
str_FileName_R = str_DataPath+'Sujeto35_R_processed.fif'
epochs_MF = mne.read_epochs(str_FileName_MF, preload=True)
epochs_R = mne.read_epochs(str_FileName_R, preload=True)
#num_sujeto = '16'

#mne.export.export_raw("Sujeto35_MF_processed.edf", epochs_MF)
#epochs_MF.plot()

CORRELATION MATRIX FOR THE MEAN OF ALL EPOCHS

In [ ]:

def calculate_mean_adjacency(epochs):
    
    m_Data = epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
    n_epochs, n_channels, n_times = m_Data.shape

    adjacency_matrices = np.zeros((n_epochs, n_channels, n_channels))

    # Loop through epochs to calculate adjacency matrix for each epoch
    for i_epoch in range(n_epochs):
        print(f'Processing epoch {i_epoch + 1} of {n_epochs}')
        
        # Initialize correlation matrix for this epoch
        correlation_matrix = np.zeros((n_channels, n_channels))
        
        # Loop through each pair of channels to calculate Pearson correlation
        for i_chan1 in range(n_channels):
            for i_chan2 in range(i_chan1, n_channels):
                # Get time series data for the two channels in this epoch
                time_series_1 = m_Data[i_epoch, i_chan1, :]
                time_series_2 = m_Data[i_epoch, i_chan2, :]
                
                # Calculate Pearson correlation
                correlation_value = np.abs(pearsonr(time_series_1, time_series_2)[0])
                
                # Update correlation matrix (symmetric matrix)
                correlation_matrix[i_chan1, i_chan2] = correlation_value
                correlation_matrix[i_chan2, i_chan1] = correlation_value
        
        # Store the correlation matrix as adjacency matrix for this epoch
        adjacency_matrices[i_epoch] = correlation_matrix

    # Calculate the mean adjacency matrix across all epochs
    mean_adjacency_matrix = np.mean(adjacency_matrices, axis=0)
    return mean_adjacency_matrix, epochs.ch_names


CORRELATION MATRIX FOR ALL EPOCHS FOR EACH FREQ BAND

In [ ]:
def calculate_bandwise_mean_adjacency(epochs, freq_bands):

    ch_names = epochs.ch_names
    band_adjacency_matrices = {}

    for band, (fmin, fmax) in freq_bands.items():
        print(f'Processing {band} band ({fmin}-{fmax} Hz)')

        # Filter the data for the specific band
        filtered_epochs = epochs.copy().filter(fmin, fmax, fir_design='firwin', verbose=False)
        m_Data = filtered_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)

        n_epochs, n_channels, _ = m_Data.shape
        adjacency_matrices = np.zeros((n_epochs, n_channels, n_channels))

        # Compute correlation for each epoch
        for i_epoch in range(n_epochs):
            correlation_matrix = np.zeros((n_channels, n_channels))

            for i_chan1 in range(n_channels):
                for i_chan2 in range(i_chan1, n_channels):
                    time_series_1 = m_Data[i_epoch, i_chan1, :]
                    time_series_2 = m_Data[i_epoch, i_chan2, :]

                    correlation_value = np.abs(pearsonr(time_series_1, time_series_2)[0])

                    correlation_matrix[i_chan1, i_chan2] = correlation_value
                    correlation_matrix[i_chan2, i_chan1] = correlation_value

            adjacency_matrices[i_epoch] = correlation_matrix

        # Compute mean adjacency matrix for this band
        band_adjacency_matrices[band] = np.mean(adjacency_matrices, axis=0)

    return band_adjacency_matrices, ch_names

NORMALIZE DATA

In [ ]:
import mne
import numpy as np

def normalizeMF(epochs_MF, epochs_R):
    """
    Normalize MF data per channel based on the mean and std of the R data per channel.
    """
    # Get the data from R and MF epochs (shape: n_epochs, n_channels, n_times)
    m_Data_MF = epochs_MF.get_data()
    m_Data_R = epochs_R.get_data()

    # Calculate the mean and std of the R data (across epochs and time) for each channel
    mean_R = np.mean(m_Data_R, axis=(0, 2))  # Mean across epochs and time for each channel
    std_R = np.std(m_Data_R, axis=(0, 2))    # Std across epochs and time for each channel

    # Initialize an array to store normalized MF data
    normalized_MF = np.zeros_like(m_Data_MF)
    normalized_R = np.zeros_like(m_Data_R)

    # Loop through each channel to normalize the data
    for i in range(m_Data_MF.shape[1]):  # Loop through channels
        # Normalize the MF data for this channel
        normalized_MF[:, i, :] = (m_Data_MF[:, i, :] - mean_R[i]) / std_R[i]

    for i in range(m_Data_R.shape[1]):  # Loop through channels
        # Normalize the MF data for this channel
        normalized_R[:, i, :] = (m_Data_R[:, i, :] - mean_R[i]) / std_R[i]

    return normalized_MF, normalized_R


FREQ BAND MATRIX FOR ALL PATIENTS

In [ ]:
'''
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from scipy.stats import pearsonr

def calculate_bandwise_mean_adjacency(epochs, fmin, fmax):
    """Compute the mean correlation (adjacency) matrix for a given frequency band."""
    ch_names = epochs.ch_names
    
    # Filter the epochs to the desired frequency band.
    filtered_epochs = epochs.copy().filter(fmin, fmax, fir_design='firwin', verbose=False)
    m_Data = filtered_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
    n_epochs, n_channels, _ = m_Data.shape

    # Compute correlation matrices for each epoch
    adjacency_matrices = np.zeros((n_epochs, n_channels, n_channels))
    for i_epoch in range(n_epochs):
        correlation_matrix = np.zeros((n_channels, n_channels))
        for i_chan1 in range(n_channels):
            for i_chan2 in range(i_chan1, n_channels):
                ts1 = m_Data[i_epoch, i_chan1, :]
                ts2 = m_Data[i_epoch, i_chan2, :]
                correlation_value = np.abs(pearsonr(ts1, ts2)[0])
                correlation_matrix[i_chan1, i_chan2] = correlation_value
                correlation_matrix[i_chan2, i_chan1] = correlation_value
        adjacency_matrices[i_epoch] = correlation_matrix

    # Compute the mean correlation matrix over epochs for this band.
    return np.mean(adjacency_matrices, axis=0), ch_names


# List of patient identifiers (35 subjects)
patients = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 
            '11', '13', '14', '15', '16', '17', '18', '19', '20',
            '21', '22', '23', '24', '25', '26', '27', '28', '29', 
            '30', '31', '32', '33', '34', '36', '37']

# Path to the preprocessed data.
str_DataPath = 'C:/Users/Sebastian/Documents/NeuroZenProcessing/Preprocessed/'

# Define frequency bands
FREQ_BANDS = {
    "Delta": (0.3, 4),
    "Theta": (4, 8),
    "Alpha": (8, 12),
    "Beta": (12, 18),
    "Fast Beta": (18, 30)
}

# Folder to save the resulting heatmap plots.
save_folder = 'C:/Users/Sebastian/Documents/NeuroZenProcessing/images'
os.makedirs(save_folder, exist_ok=True)

# Process one frequency band at a time
for band, (fmin, fmax) in FREQ_BANDS.items():
    print(f"\nProcessing {band} band ({fmin}-{fmax} Hz)")

    all_adjacency_MF = []
    all_adjacency_R = []

    # Process each patient
    for num_sujeto in patients:
        print(f"Processing Sujeto{num_sujeto}")

        str_FileName_MF = os.path.join(str_DataPath, f"Sujeto{num_sujeto}_MF_processed.fif")
        str_FileName_R = os.path.join(str_DataPath, f"Sujeto{num_sujeto}_R_processed.fif")

        # Load the epochs
        epochs_MF = mne.read_epochs(str_FileName_MF, preload=True)
        epochs_R = mne.read_epochs(str_FileName_R, preload=True)

        # Normalize the data (using resting data statistics)
        normalized_MF, normalized_R = normalizeMF(epochs_MF, epochs_R)

        # Convert normalized arrays back into mne.EpochsArray objects
        normalized_MF = mne.EpochsArray(normalized_MF, epochs_MF.info)
        normalized_R = mne.EpochsArray(normalized_R, epochs_R.info)

        # Compute bandwise mean adjacency matrices
        adjacencies_MF, tick_labels = calculate_bandwise_mean_adjacency(normalized_MF, fmin, fmax)
        adjacencies_R, _ = calculate_bandwise_mean_adjacency(normalized_R, fmin, fmax)

        # Store the subject's adjacency matrices
        all_adjacency_MF.append(adjacencies_MF)
        all_adjacency_R.append(adjacencies_R)

    # Compute mean adjacency matrices across all patients for the current band
    mean_adjacency_MF = np.mean(all_adjacency_MF, axis=0)
    mean_adjacency_R = np.mean(all_adjacency_R, axis=0)
    difference_matrix = mean_adjacency_MF - mean_adjacency_R

    print(f"Generating plot for {band} band")

    plt.figure(figsize=(35, 12))

    # Plot for Mindfulness condition
    plt.subplot(1, 3, 1, aspect='equal')
    sn.heatmap(pd.DataFrame(mean_adjacency_MF, columns=tick_labels, index=tick_labels),
               xticklabels=2, yticklabels=2, cmap='jet', cbar_kws={'shrink': 0.4})
    plt.title("Mindfulness", fontdict={'size': 12, 'weight': 'bold'})

    # Plot for Resting condition
    plt.subplot(1, 3, 2, aspect='equal')
    sn.heatmap(pd.DataFrame(mean_adjacency_R, columns=tick_labels, index=tick_labels),
               xticklabels=2, yticklabels=2, cmap='jet', cbar_kws={'shrink': 0.4})
    plt.title("Resting", fontdict={'size': 12, 'weight': 'bold'})

    # Plot for the Difference (MF - R)
    plt.subplot(1, 3, 3, aspect='equal')
    sn.heatmap(pd.DataFrame(difference_matrix, columns=tick_labels, index=tick_labels),
               xticklabels=2, yticklabels=2, cmap='jet', cbar_kws={'shrink': 0.4}, center=0)
    plt.title("Difference (MF - R)", fontdict={'size': 12, 'weight': 'bold'})

    plt.suptitle(f"Correlation Matrices for {band} Band", fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.subplots_adjust(top=0.99)

    save_path = os.path.join(save_folder, f"{band}_CorrelationMatrices.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Saved plot for {band} band to {save_path}")
    plt.close()

'''

MATRIX FOR ALL PATIENTS

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# Function to calculate mean adjacency matrix for all patients
def calculate_mean_adjacency_across_patients(normalized_MF_list, normalized_R_list):
    """
    Calculate the mean adjacency matrix across multiple patients.
    """
    mean_adjacency_MF_list = []
    mean_adjacency_R_list = []
    num_sujeto = 0
    for normalized_MF, normalized_R in zip(normalized_MF_list, normalized_R_list):
        adjacency_MF, tick_labels = calculate_mean_adjacency(normalized_MF)
        adjacency_R, _ = calculate_mean_adjacency(normalized_R)

        np.save(f"C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/Matrices/Sujeto{patients[num_sujeto]}_MF.npy", adjacency_MF)
        np.save(f"C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/Matrices/Sujeto{patients[num_sujeto]}_R.npy", adjacency_R)
        print(f"Saved Sujeto{num_sujeto} correlation matrices")
        mean_adjacency_MF_list.append(adjacency_MF)
        mean_adjacency_R_list.append(adjacency_R)
        num_sujeto += 1

    # Calculate mean adjacency matrices across all patients
    mean_adjacency_MF = np.mean(mean_adjacency_MF_list, axis=0)
    mean_adjacency_R = np.mean(mean_adjacency_R_list, axis=0)

    return mean_adjacency_MF, mean_adjacency_R, tick_labels

# Process all patients
normalized_MF_list = []
normalized_R_list = []

patients = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '13', '14', '15', '16', '17', '18', '19', '20',
            '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '36', '37'] 

print(patients)

for num_sujeto in patients:
    print(f"Processing Sujeto{num_sujeto} for mean adjacency matrices")
    str_FileName_MF = str_DataPath + f"Sujeto{num_sujeto}_MF_processed.fif"
    str_FileName_R = str_DataPath + f"Sujeto{num_sujeto}_R_processed.fif"

    # Load data
    normalized_MF = mne.read_epochs(str_FileName_MF, preload=True)
    normalized_R = mne.read_epochs(str_FileName_R, preload=True)
    
    epochs_info = normalized_MF.info
    # Normalize data
    normalized_MF, normalized_R = normalizeMF(normalized_MF, normalized_R)

    # Convert to EpochsArray
    normalized_MF = mne.EpochsArray(normalized_MF, epochs_info)
    normalized_R = mne.EpochsArray(normalized_R, epochs_info)

    # Append to list
    normalized_MF_list.append(normalized_MF)
    normalized_R_list.append(normalized_R)

# Calculate mean adjacency matrices across patients
mean_adjacency_MF, mean_adjacency_R, tick_labels = calculate_mean_adjacency_across_patients(
    normalized_MF_list, normalized_R_list)

# Difference matrix
difference_matrix = mean_adjacency_MF - mean_adjacency_R
np.save("C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/Matrices/ALL_MF_matrix.npy", mean_adjacency_MF)
np.save("C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/Matrices/ALL_R_matrix.npy", mean_adjacency_R)
np.save("C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/Matrices/ALL_difference_matrix.npy", difference_matrix)
# Plot mean adjacency matrices
plt.figure(figsize=(35, 12))

plt.subplot(1, 3, 2, aspect='equal')
sn.heatmap(pd.DataFrame(mean_adjacency_R, columns=tick_labels, index=tick_labels), 
           xticklabels=2, yticklabels=2, cmap='jet', cbar_kws={'shrink': 0.4})
plt.title("Resting", fontdict={'size': 12, 'weight': 'bold'})

plt.subplot(1, 3, 1, aspect='equal')
sn.heatmap(pd.DataFrame(mean_adjacency_MF, columns=tick_labels, index=tick_labels), 
           xticklabels=2, yticklabels=2, cmap='jet', cbar_kws={'shrink': 0.4})
plt.title("Mindfulness", fontdict={'size': 12, 'weight': 'bold'})

plt.subplot(1, 3, 3, aspect='equal')
sn.heatmap(pd.DataFrame(difference_matrix, columns=tick_labels, index=tick_labels), 
           xticklabels=2, yticklabels=2, cmap='jet', cbar_kws={'shrink': 0.4}, center=0)
plt.title("Difference (MF - R)", fontdict={'size': 12, 'weight': 'bold'})

plt.suptitle("Correlation Matrices for 10 Participants", 
             fontsize=16, fontweight='bold')

plt.tight_layout()
plt.subplots_adjust(top=0.99)

# Save plot
save_folder = 'C:/Users/Sebastian/Documents/NeuroZenProcessing/images'
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder, "CorrelationMatrixAllParticipants.png")
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"Saved mean adjacency matrix plot to {save_path}")
plt.close()



GRAPHS

In [ ]:
#!pip install python-louvain
#!pip install networkx
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import scipy.cluster.hierarchy as sch
from community import community_louvain
import os

# Convert adjacency matrices to graphs
def create_graph(adjacency_matrix, threshold=0.7, node_labels=None):
    """
    Create an undirected graph from an adjacency matrix, applying a threshold.

    Parameters:
    - adjacency_matrix (np.array): The adjacency matrix.
    - threshold (float): Minimum weight for an edge to be included.
    - node_labels (list): List of node labels corresponding to the matrix indices.

    Returns:
    - G (networkx.Graph): The constructed graph.
    """
    G = nx.Graph()

    if node_labels is None:
        node_labels = [str(i) for i in range(len(adjacency_matrix))]  # Default to numbers if labels are missing

    # Ensure node_labels matches adjacency_matrix size
    assert len(node_labels) == len(adjacency_matrix), "Node labels must match matrix size!"

    # Add nodes using real labels
    for i, label in enumerate(node_labels):
        G.add_node(label)  # Use labels instead of numbers

    # Add edges based on threshold
    for i in range(len(adjacency_matrix)):
        for j in range(i + 1, len(adjacency_matrix)):  # Avoid duplicate edges
            if adjacency_matrix[i, j] >= threshold:
                G.add_edge(node_labels[i], node_labels[j], weight=adjacency_matrix[i, j])  # Use labels

    return G

# Compute graph metrics
def compute_graph_metrics(G, nrand=50):
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    avg_degree = np.mean([deg for _, deg in G.degree()])
    clustering_coeff = nx.average_clustering(G)

    # Handle shortest path length only if graph is connected
    avg_shortest_path = nx.average_shortest_path_length(G) if nx.is_connected(G) else np.nan  

    C_rand_list, L_rand_list = [], []
    for _ in range(nrand):
        random_graph = nx.gnm_random_graph(num_nodes, num_edges)
        
        if nx.is_connected(random_graph):  # Only use connected random graphs
            C_rand_list.append(nx.average_clustering(random_graph))
            L_rand_list.append(nx.average_shortest_path_length(random_graph))

    C_rand = np.mean(C_rand_list) if C_rand_list else np.nan
    L_rand = np.mean(L_rand_list) if L_rand_list else np.nan

    small_worldness = (clustering_coeff / C_rand) / (avg_shortest_path / L_rand) if not np.isnan(C_rand) and not np.isnan(L_rand) and not np.isnan(avg_shortest_path) else np.nan

    centrality = np.mean(list(nx.betweenness_centrality(G).values()))
    modularity = community_louvain.modularity(community_louvain.best_partition(G), G)

    return {
        "num_nodes": num_nodes,
        "num_edges": num_edges,
        "average_degree": avg_degree,
        "clustering_coefficient": clustering_coeff,
        "average_shortest_path": avg_shortest_path,
        "small_worldness": small_worldness,
        "betweenness_centrality": centrality,
        "modularity": modularity
    }


# Visualize graph
def visualize_graph(G, title, save_path):
    """
    Visualize a network graph.

    Parameters:
    - G (networkx.Graph): The input graph.
    - title (str): Title of the graph.
    - save_path (str): Path to save the figure.
    """
    plt.figure(figsize=(10, 10))

    pos = nx.spring_layout(G, seed=42)  # Position nodes
    nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', edge_color='gray', font_size=8)

    plt.title(title)
    plt.savefig(save_path, dpi=300)
    print(f"Graph saved to {save_path}")
    plt.close()

patients = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '13', '14', '15', '16', '17', '18', '19', '20',
            '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '36', '37'] 
all_metrics = []

data_dir = "C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/Matrices"
output_csv = "C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/All_Graph_Metrics.csv"

for num_sujeto in patients:
    print(f"Processing Sujeto{num_sujeto}")
    
    for state in ["MF", "R"]:
        file_path = os.path.join(data_dir, f"Sujeto{num_sujeto}_{state}.npy")
        adjacency_matrix = np.load(file_path)
        
        G = create_graph(adjacency_matrix, threshold=0.7)
        metrics = compute_graph_metrics(G, nrand=50)
        visualize_graph(G, f"{state} Graph", f"C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/Graphs/Sujeto{num_sujeto}_{state}_Graph.png")
        metrics["Participant"] = f"{num_sujeto}_{state}"
        all_metrics.append(metrics)

df_metrics = pd.DataFrame(all_metrics)
df_metrics.to_csv(output_csv, index=False)
print(f"All graph metrics saved to {output_csv}")


In [ ]:
import pandas as pd
from scipy.stats import ttest_rel

# Load CSV
csv_path = "C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/All_Graph_Metrics.csv"
df = pd.read_csv(csv_path)

# Extract MF and R data
df["Participant_ID"] = df["Participant"].str.split("_").str[0]  # Extract participant number
df_MF = df[df["Participant"].str.endswith("_MF")].set_index("Participant_ID")
df_R = df[df["Participant"].str.endswith("_R")].set_index("Participant_ID")

# Ensure data aligns correctly
df_MF = df_MF.sort_index()
df_R = df_R.sort_index()

# Get metric columns (excluding "Participant" and "Participant_ID")
metric_columns = [col for col in df.columns if col not in ["Participant", "Participant_ID", "num_nodes"]]

# Perform paired t-tests
t_test_results = {}
for metric in metric_columns:
    # Drop NaN values from both MF and R before testing
    valid_data = df_MF[[metric]].join(df_R[[metric]], lsuffix="_MF", rsuffix="_R").dropna()
    
    if not valid_data.empty:  # Ensure there's data left to test
        t_stat, p_value = ttest_rel(valid_data[f"{metric}_MF"], valid_data[f"{metric}_R"])
        t_test_results[metric] = {"t-statistic": t_stat, "p-value": p_value}
    else:
        t_test_results[metric] = {"t-statistic": None, "p-value": None}  # No valid data

# Convert results to DataFrame and save
df_ttest = pd.DataFrame.from_dict(t_test_results, orient="index")
ttest_csv_path = "C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/TTest_Results.csv"
df_ttest.to_csv(ttest_csv_path)

print(f"T-test results saved to {ttest_csv_path}")
print(df_ttest)


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

data_dir = "C:/Users/Sebastian/Documents/NeuroZenProcessing/Connectivity_Outputs/Matrices"
num_sujeto = '9'
state = "R"  # or "R"
file_path = os.path.join(data_dir, f"Sujeto{num_sujeto}_{state}.npy")
adjacency_matrix = np.load(file_path)


def plot_corr_matrix(corr_matrix, labels, title='Reordered Correlation Matrix', cmap='coolwarm'):
    fig, ax = plt.subplots(figsize=(10, 8))
    cax = ax.matshow(corr_matrix, cmap=cmap, vmin=0, vmax=1)
    fig.colorbar(cax)

    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels, rotation=90, fontsize=6)
    ax.set_yticklabels(labels, fontsize=6)

    plt.title(title, y=1.15)
    plt.tight_layout()
    plt.show()

original_labels = ['Fp1', 'Fpz','Fp2','F7','F3','Fz','F4','F8','FC5','FC1','FC2','FC6','T7','C3','Cz','C4','T8','CP5','CP1','CP2',
                'CP6','P7','P3','Pz','P4','P8','POz','O1','O2','AF7','AF3','AF4','AF8','F5','F1','F2','F6','FC3','FCz','FC4',
              'C5','C1','C2','C6','CP3','CP4','P5','P1','P2','P6','PO5','PO3','PO4','PO6','FT7','FT8','TP7','TP8','PO7','PO8','Oz']
ordered_labels = [
    'Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FT8', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6',
    'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
    'TP7', 'TP8', 'CP5', 'CP3', 'CP1', 'CP2', 'CP4', 'CP6',
    'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8',
    'PO7', 'PO5', 'PO3', 'POz', 'PO4', 'PO6', 'PO8',
    'O1', 'Oz', 'O2'
]

# ---- 4. Reorder the matrix and labels ----
label_to_index = {label: i for i, label in enumerate(original_labels)}
reorder_idx = [label_to_index[label] for label in ordered_labels]
reordered_matrix = adjacency_matrix[np.ix_(reorder_idx, reorder_idx)]

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# --- Left: Original Matrix ---
df_original = pd.DataFrame(adjacency_matrix, index=original_labels, columns=original_labels)
sn.heatmap(df_original, cmap='jet', xticklabels=2, yticklabels=2, 
           cbar_kws={'shrink': 0.5}, ax=axes[0], square=True, vmin=0, vmax=1)
axes[0].set_title("Original Correlation", fontsize=12, fontweight='bold')

# --- Right: Reordered Matrix ---
df_reordered = pd.DataFrame(reordered_matrix, index=ordered_labels, columns=ordered_labels)
sn.heatmap(df_reordered, cmap='jet', xticklabels=2, yticklabels=2, 
           cbar_kws={'shrink': 0.5}, ax=axes[1], square=True, vmin=0, vmax=1)
axes[1].set_title("Reordered by Brain Region", fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()